In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [14]:
# Initialize the SentenceTransformer model
model=SentenceTransformer('all-MiniLM-L6-v2')

# Sample text
text="""
Langchain is a framework for building applications with LLMs
Langchain provides modular abstractions to combine LLMs with tools like openai and Pinecone.
You can create chains, agents, memory and retrievers.
The Eiffel tower is located in Paris.
France is a popular tourist destination.
"""

#Step1: Split the text into sentences
sentences = [sentence.strip() for sentence in text.split('\n') if sentence]

#Step2: Generate embeddings for each sentence
embeddings = model.encode(sentences)

#Step3: Initialize variables for semantic chunking
threshold = 0.7  # Similarity threshold
chunks = []
current_chunk = [sentences[0]]

#Step4: Perform semantic chunking based on cosine similarity
for i in range(1, len(sentences)):
    sim = cosine_similarity([embeddings[i]], [embeddings[i-1]])[0][0]
    if sim >= threshold:
        current_chunk.append(sentences[i])
        
    else:
        chunks.append(" ".join(current_chunk))
        current_chunk = [sentences[i]]

# Append the last chunk
chunks.append(" ".join(current_chunk))

# Output the semantic chunks
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx+1}: {chunk}\n")

Chunk 1: Langchain is a framework for building applications with LLMs Langchain provides modular abstractions to combine LLMs with tools like openai and Pinecone.

Chunk 2: You can create chains, agents, memory and retrievers.

Chunk 3: The Eiffel tower is located in Paris.

Chunk 4: France is a popular tourist destination.



### RAG PIPELINE

In [46]:
from dotenv import load_dotenv
import os

# Load your .env file (this populates os.environ automatically)
load_dotenv()

# Now your previous code works without changes
from langchain_core.output_parsers import StrOutputParser
# os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  # DELETE THIS LINE

# Verify it loaded
print("GROQ_API_KEY loaded:", bool(os.getenv("GROQ_API_KEY")))  # Should print True


GROQ_API_KEY loaded: True


In [47]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

In [48]:
## Custom Semantic Chunking with Threshold

class ThresholdSemanticChunker:
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2', threshold: float = 0.7):
        self.model = SentenceTransformer(model_name)
        self.threshold = threshold

    def _call(self, text: str) -> list[str]:
        sentences = [sentence.strip() for sentence in text.split('\n') if sentence]
        embeddings = self.model.encode(sentences)

        chunks = []
        current_chunk = [sentences[0]]

        for i in range(1, len(sentences)):
            sim = cosine_similarity([embeddings[i]], [embeddings[i-1]])[0][0]
            if sim >= self.threshold:
                current_chunk.append(sentences[i])
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentences[i]]

        chunks.append(" ".join(current_chunk))
        return chunks

    def split_documents(self, docs):
        results = []
        for doc in docs:
            for chunk in self._call(doc.page_content):
                results.append(Document(page_content=chunk, metadata=doc.metadata))
        return results

In [49]:
# Sample text
sample_text="""
Langchain is a framework for building applications with LLMs
Langchain provides modular abstractions to combine LLMs with tools like openai and Pinecone.
You can create chains, agents, memory and retrievers.
The Eiffel tower is located in Paris.
France is a popular tourist destination.
"""

doc = Document(page_content=sample_text, metadata={})

In [50]:
chunker = ThresholdSemanticChunker(threshold=0.7)
chunks = chunker.split_documents([doc])
chunks

[Document(metadata={}, page_content='Langchain is a framework for building applications with LLMs Langchain provides modular abstractions to combine LLMs with tools like openai and Pinecone.'),
 Document(metadata={}, page_content='You can create chains, agents, memory and retrievers.'),
 Document(metadata={}, page_content='The Eiffel tower is located in Paris.'),
 Document(metadata={}, page_content='France is a popular tourist destination.')]

In [59]:
## Vector Store Integration
from langchain_huggingface import HuggingFaceEmbeddings

# Use proper LangChain embeddings wrapper
embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device': 'cpu'}  # Use 'cuda' if GPU available
)


vector_store = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x306d72210>, search_kwargs={})

In [60]:
## Prompt Template

prompt_template = """ You are a knowledgeable assistant. Use the following context to answer the question:

Context: {context}

Answer the question: {question}

"""
prompt = PromptTemplate.from_template(prompt_template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=' You are a knowledgeable assistant. Use the following context to answer the question:\n\nContext: {context}\n\nAnswer the question: {question}\n\n')

In [65]:
### LLM
llm = init_chat_model(model="groq:llama-3.3-70b-versatile", temperature=0.4, api_key=os.getenv("GROQ_API_KEY"))
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x306f46190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x306f45480>, model_name='llama-3.3-70b-versatile', temperature=0.4, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [66]:
# LCEL
rag_chain = (
    RunnableMap(
    {
        "context": RunnableLambda(
            lambda x: retriever.invoke(x["question"]),
        ),
        "question": RunnableLambda(lambda x: x["question"]),
    }
)

| prompt
| llm
| StrOutputParser()
)


In [67]:
# Example Query
query = "Where is the Eiffel tower located?"
response = rag_chain.invoke({"question": query})
print("Response:", response)

Response: The Eiffel tower is located in Paris.


In [69]:
# Example Query
query = "Where is the Great Wall of China located?"
response = rag_chain.invoke({"question": query})
print("Response:", response)

Response: The provided context does not mention the location of the Great Wall of China. The context only discusses the Eiffel Tower, France, and the Langchain framework, but does not provide any information about the Great Wall of China. Therefore, I cannot answer the question based on the given context.


### Semantic Chunker with Langchain

In [73]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import TextLoader

In [74]:
loader = TextLoader("langchain_intro.txt")
docs = loader.load()

embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device': 'cpu'}  # Use 'cuda' if GPU available
)

chunker = SemanticChunker(embeddings=embeddings)
chunks = chunker.split_documents(docs)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk.page_content}\n")

Chunk 1: Langchain is a framework for building applications with LLMs
Langchain provides modular abstractions to combine LLMs with tools like openai and Pinecone. You can create chains, agents, memory and retrievers.

Chunk 2: The Eiffel tower is located in Paris. France is a popular tourist destination.

